In [2]:
import sys
sys.path.append("../src")

from nat2324.algorithms import ParticleSwarmOptimization as PSO
from nat2324.problems.objectives import Objectives

%load_ext autoreload
%autoreload 2

In [32]:
objective = Objectives(function_type="rastrigin")
pso = PSO(fitness_fn=objective.evaluate, bounds=(-5.12, 5.12), D=20, alpha3=2)
pso.run(patience=None, max_generations=1000)

Current best 597.92186830: 100%|██████████| 1000/1000 [00:01<00:00, 870.72it/s]


array([-5.12, -5.12, -5.12, -5.12,  5.12,  5.12, -5.12,  5.12, -5.12,
       -5.12,  5.12,  5.12, -5.12,  5.12,  5.12, -5.12, -5.12, -5.12,
        5.12,  5.12])

In [35]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def visualize_functions(objectives, max_col=8):
    function_names = [func for func in dir(objectives) if callable(getattr(objectives, func)) and not func.startswith("__") and func != '__weakref__']
    print(function_names)
    return
    n = len(function_names)
    n_cols = min(n, max_col)
    n_rows = n // n_cols + (n % n_cols > 0)

    fig = plt.figure(figsize=(n_cols * 4, n_rows * 4))

    for i, function_name in enumerate(function_names, 1):
        ax = fig.add_subplot(n_rows, n_cols, i, projection='3d')
        x = y = np.linspace(-5, 5, 100)
        X, Y = np.meshgrid(x, y)
        Z = getattr(objectives, function_name)(np.array([X, Y]))
        ax.plot_surface(X, Y, Z, rstride=1, cstride=1,
                        cmap='viridis', edgecolor='none')
        ax.set_title(function_name)

    plt.tight_layout()
    plt.show()

objectives = Objectives()
visualize_functions(objectives)


TypeError: descriptor '__weakref__' for 'Objectives' objects doesn't apply to a 'Objectives' object